In [1]:
from vision import Vision
import cv2

VISION

In [ ]:
# Main function to test the Vision class
if __name__ == "__main__":
    try:
        vision = Vision(target_height=10)
        
        
        vision.update()
        matrix = vision.get_matrix()
        print("Color Matrix:")
        print(matrix)



        # while True:                                           capture images in loop
        #     # Update the vision system
        #     vision.update()

        #     # Get the current matrix and display it
        #     matrix = vision.get_matrix()
        #     print("Color Matrix:")
        #     print(matrix)

        #     # Display the current image
        #     cv2.imshow("Captured Image", vision.get_image())    
    finally:
        vision.release()


GLOBAL PLANNING

LOCAL PLANNING

FLITERING

MOTION CONTROL